<a href="https://colab.research.google.com/github/aikssa/python_do_zero_ao_ds/blob/main/aula_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análise de preços de casas - Aula 04

*Importante: para visualizar os mapas e gráficos dinâmicos, clique em <img src="https://raw.githubusercontent.com/aikssa/aikssa/main/img/colab.png">.*

# Importação das bibliotecas e dos dados

In [1]:
# importando as bibliotecas necessárias
import pandas as pd
import numpy as np
import plotly.express as px
from geopy.geocoders import Nominatim
import ipywidgets as widgets
from ipywidgets import fixed
import matplotlib.pyplot as plt
from matplotlib import gridspec
import seaborn as sns
from tqdm import tqdm

In [2]:
# carregando o dataset
df = pd.read_csv('/content/drive/MyDrive/repos/python_do_zero_ao_ds/datasets/kc_house_data.csv')

# verificando as 5 primeiras linhas do dataframe
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


# Respondendo questões de negócio

## Questões dadas em aula

### Q1: Qual a quantidade de imóveis por nível?

- Nivel 0: Preço entre 0.00 e 321.950
- Nivel 1: Preço entre 321.950 e 450.000
- Nivel 2: Preço entre 450.000 e 645.000
- Nivel 3: Preço acima de 645.000

In [3]:
# criando uma nova coluna e definindo o nivel dos imóveis
df['level'] = df['price'].apply(lambda x: 0 if x < 321950 else 1 if x < 450000 else 2 if x < 645000 else 3)

In [4]:
# verificando a quantidade de imóveis por nível
df['level'].value_counts()

2    5508
3    5413
0    5403
1    5289
Name: level, dtype: int64

**Resposta**: 
- 5403 imóveis são do nível 0
- 5289 imóveis são do nível 1
- 5508 imóveis são do nível 2
- 5413 imóveis são do nível 3

### Q2: Adicione as seguintes informações ao imóvel:

- Nome da Rua
- Número do imóvel
- O nome do Bairro
- O nome da cidade
- O nome do estado

Para resolver essa questão, utilizaremos a biblioteca [Geopy](https://geopy.readthedocs.io/en/stable/#) (já importada acima).

Tendo em vista que o nosso dataframe possui 21613 linhas e seria exigida uma certa capacidade de processamento para adicionar as informações solicitadas a todas elas, vamos pegar as 50 primeiras linhas para atender à solicitação do CEO.

Vamos utilizar também o [TQDM](https://github.com/tqdm/tqdm) para incluir uma barra de progresso.

In [5]:
# criando um novo dataframe com as 50 primeiras linhas 
df1 = df[:50].copy()

# verificando o dataframe
df1.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,level
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650,0
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639,2
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062,0
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000,2
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503,2


In [6]:
# criando as colunas vazias
df1['road'] = 'NA'
df1['house_number'] = 'NA'
df1['neighbourhood'] = 'NA'
df1['city'] = 'NA'
df1['state'] = 'NA'

# instanciando a api do nominatim
geolocator = Nominatim(user_agent='geoapiExercise')

# utilizando um loop for para preencher os dados
for i in tqdm(range(len(df1))):
    # concatenando lat e long
    query = str(df1.loc[i, 'lat']) + ',' + str(df1.loc[i, 'long'])

    # criando uma variável para receber lat e long
    response = geolocator.reverse(query)

    # verificando se existe o dado na lista retornada pela api e preenchendo os dados
    if 'road' in response.raw['address']:
        df1.loc[i, 'road'] = response.raw['address']['road']
    
    if 'house_number' in response.raw['address']:
        df1.loc[i, 'house_number'] = response.raw['address']['house_number']

    if 'neighbourhood' in response.raw['address']:
        df1.loc[i, 'neighbourhood'] = response.raw['address']['neighbourhood']

    if 'city' in response.raw['address']:
        df1.loc[i, 'city'] = response.raw['address']['city']

    if 'state' in response.raw['address']:
        df1.loc[i, 'state'] = response.raw['address']['state']

100%|██████████| 50/50 [00:24<00:00,  2.02it/s]


In [7]:
# verificando o resultado
df1[['id', 'lat', 'long', 'road', 'house_number', 'neighbourhood', 'city', 'state']].sample(5)

,id,lat,long,road,house_number,neighbourhood,city,state
37,2768000400,47.6702,-122.362,Northwest 58th Street,NA,NA,Seattle,Washington
0,7129300520,47.5112,-122.257,61st Avenue South,10012,Rainier Beach,Seattle,Washington
49,822039084,47.4041,-122.451,NA,NA,NA,NA,Washington
2,5631500400,47.7379,-122.233,81st Avenue Northeast,15109,NA,NA,Washington
22,7137970340,47.3266,-122.169,Southeast 307th Place,12840,NA,NA,Washington


### Q3: Adicione o nível do imóvel no mapa como uma cor.

### Q4: Adicione o preço do imóvel como o tamanho do ponto no mapa.

Como as questões 3 e 4 se complementam, vamos plotar um mapa no qual o nível será diferenciado pela cor e o preço pelo tamanho do ponto.

In [8]:
# selecionando as variáveis que serão utilizadas para plotar o mapa
df_mapa = df[['id', 'price', 'lat', 'long', 'level']]

# plotando o mapa
mapa = px.scatter_mapbox(data_frame=df_mapa,
                         lat='lat',
                         lon='long',
                         color='level',
                         hover_name='id',
                         hover_data = ['price'],
                         size='price',
                         color_continuous_scale=px.colors.sequential.Viridis_r,
                         size_max=30,
                         zoom=10,
                         mapbox_style='open-street-map',
                         height=600)

# definindo a margem do mapa
mapa.update_layout(margin={'r':0, 't':0, 'l':0, 'b':0})

# plotando o mapa
mapa.show()

### Q5: Adicione opções de filtros para fazer as próprias análises:
- Eu quero escolher visualizar imóveis com vista para água ou não
- Eu quero escolher visualizar imóveis até um certo valor de preço

In [9]:
# criando uma nova coluna para verificar se a casa tem ou não vista para a água
df['is_waterfront'] = df['waterfront'].apply(lambda x: 'yes' if x == 1 else 'no')

In [10]:
# definindo o estilo do botão
style = {'description_width': 'initial'}

# criando os botões interativos
price_limit = widgets.IntSlider(value=540000, min=75000, max=7700000, description='Preço', disabled=False, style=style)

waterfront_radio = widgets.RadioButtons(options=df['is_waterfront'].unique().tolist(), value='yes', 
                                        description='Water view', disabled=False, style=style)

# criando uma função para plotar o mapa
def update_map(df, waterfront, limit):
    houses = df[(df['price'] <= limit) & (df['is_waterfront'] == waterfront)][['id', 'lat', 'long', 'price', 'level']]

    mapa = px.scatter_mapbox(data_frame=houses,
                            lat='lat',
                            lon='long',
                            color='level',
                            size='price',
                            color_continuous_scale=px.colors.sequential.Viridis_r,
                            size_max=15,
                            zoom=10)

    mapa.update_layout(mapbox_style='open-street-map')
    mapa.update_layout(height=600, margin={'r':0, 't':0, 'l':0, 'b':0})

    mapa.show()

In [11]:
# verificando o resultado da função
# o fixed determina que os dados não mudam, apenas os filtros
widgets.interactive(update_map, df=fixed(df), waterfront=waterfront_radio, limit=price_limit)

interactive(children=(RadioButtons(description='Water view', index=1, options=('no', 'yes'), style=Description…

### Q6: Adicionar opções de filtros no último dashboard enviado:
- Visualizar somente valores a partir de uma data disponível para compra

In [12]:
# definindo o estilo dos gráficos
sns.set_theme(style='whitegrid', palette='viridis')
palette = 'viridis'
title = {'size':16, 'color':'#2b2b2b'}
labels = {'size':14, 'color':'#2b2b2b'}
ticks = '#2b2b2b'

# mudando o formato da data
df['year'] = pd.to_datetime(df['date']).dt.strftime('%Y')
df['date'] = pd.to_datetime(df['date']).dt.strftime('%Y-%m-%d')
df['year_week'] = pd.to_datetime(df['date']).dt.strftime('%Y-%U')

# criando o botão para filtrar os dados
date_limit = widgets.SelectionSlider(options=df['date'].sort_values().unique().tolist(),
                                     value='2014-12-01',
                                     description='Disponível',
                                     disabled=False,
                                     continuous_update=False,
                                     orientation='horizontal',
                                     readout=True)

# criando a função para plotar o dashboard apartir do filtro
def update_dashboard(df, limit):
    df1 = df[df['date'] <= limit].copy() # filtrando os dados

    # criando o dashboard
    fig = plt.figure(figsize=(20, 12))
    specs = gridspec.GridSpec(nrows=2, ncols=2, figure=fig, hspace=0.3, wspace=0.2, width_ratios=[2, 1])
    fig.tight_layout()

    ax1 = fig.add_subplot(specs[0, :])
    ax2 = fig.add_subplot(specs[1, 0])
    ax3 = fig.add_subplot(specs[1, 1])

    # primeiro gráfico
    by_day = df1[['id', 'date']].groupby('date').mean().reset_index()
    sns.lineplot(data=by_day, x='date', y='id', ax=ax1)
    ax1.set_title('Imóveis disponíveis para compra por dia', fontdict=title, pad=20)
    ax1.set_xlabel('')
    ax1.set_ylabel('')
    ax1.tick_params(colors=ticks)
    ax1.set_xticks([df['date'].min(), limit])
    ax1.xaxis.set_major_locator(plt.MaxNLocator(10))
    ax1.ticklabel_format(style='plain', axis='y', useOffset=False)
    ax1.grid(axis='x')
    sns.despine(left=True)

    # segundo gráfico
    by_week = df1[['price', 'year_week']].groupby('year_week').mean().reset_index()
    sns.barplot(data=by_week, x='year_week', y='price', ax=ax2, palette=palette)
    ax2.set_title('Imóveis disponíveis para compra por semana', fontdict=title, pad=20)
    ax2.set_xlabel('')
    ax2.set_ylabel('')
    ax2.tick_params(colors=ticks)
    ax2.tick_params(rotation=90, axis='x')
    ax2.ticklabel_format(style='plain', axis='y', useOffset=False)

    # terceiro gráfico
    by_year = df1[['price', 'year']].groupby('year').sum().reset_index()
    sns.barplot(data=by_year, x='year', y='price', ax=ax3)
    ax3.set_title('Imóveis disponíveis para compra por ano', fontdict=title, pad=20)
    ax3.set_xlabel('')
    ax3.set_ylabel('')
    ax3.tick_params(colors=ticks)
    ax3.ticklabel_format(style='plain', axis='y', useOffset=False)

    plt.show()

widgets.interactive(update_dashboard, df=fixed(df), limit=date_limit)

interactive(children=(SelectionSlider(continuous_update=False, description='Disponível', index=212, options=('…

## Exercícios

### Q1. Qual a quantidade de imóveis por nível?
- Nivel 0: Preço entre 0.00 e 321.950
- Nivel 1: Preço entre 321.950 e 450.000
- Nivel 2: Preço entre 450.000 e 645.000
- Nivel 3: Preço acima de 645.000

Como essa questão já foi respondida anteriormente, vamos apenas fazer o código novamente.

In [13]:
# criando uma nova coluna e definindo o nivel dos imóveis
df['level'] = df['price'].apply(lambda x: 0 if x < 321950 else 
                                1 if x < 450000 else 
                                2 if x < 645000 else 3)

In [14]:
# verificando a quantidade de imóveis por nível
df['level'].value_counts()

2    5508
3    5413
0    5403
1    5289
Name: level, dtype: int64

**Resposta**: 
- 5403 imóveis são do nível 0
- 5289 imóveis são do nível 1
- 5508 imóveis são do nível 2
- 5413 imóveis são do nível 3

### Q2: Qual a média do tamanho da sala de estar dos imíveis por "size"?
- Size 0: Tamanho entre 0 e 1427 sqft
- Size 1: Tamanho entre 1427 e 1910 sqft
- Size 2: Tamanho entre 1910 e 2550 sqft
- Size 3: Tamanho acima de 2550 sqtf

In [15]:
# criando uma nova coluna e definindo o tamanho dos imóveis
df['size'] = df['sqft_living'].apply(lambda x: 0 if x < 1427 else 1 if x < 1910 else 2 if x < 2550 else 3)

In [16]:
# verificando a média do tamanho da sala de estar de acordo com o siza
print('Size 0: ', df['sqft_living'][df['size'] == 0].mean())
print('Size 1: ', df['sqft_living'][df['size'] == 1].mean())
print('Size 2: ', df['sqft_living'][df['size'] == 2].mean())
print('Size 3: ', df['sqft_living'][df['size'] == 3].mean())

Size 0:  1123.7764205071257
Size 1:  1661.1980884557722
Size 2:  2202.7002389266677
Size 3:  3318.9884041965765


**Resposta**: A média do tamanho da sala de estar dos imóveis por `size` é:
- Size 0 (entre 0 e 1427 sqft): 1123.77
- Size 1 (entre 1427 e 1910 sqft): 1661.19
- Size 2 (entre 1910 e 2550 sqft): 2202.70
- Size 3 (acima de 2550 sqtf): 3318.98

### Q3: Adicione as seguintes informações ao conjunto de dados original:
- Place ID: Identificação da localização
- OSM type: Open Street Map Type
- Country: Nome do País
- Country CODE: Código do País

Como fizemos algo parecido no exercício acima, vamos utilizar o `df1` para adicionarmos os dados solicitados.

In [17]:
# criando as colunas vazias
df1['place_id'] = 'NA'
df1['osm_type'] = 'NA'
df1['country'] = 'NA'
df1['country_code'] = 'NA'

# instanciando a api do nominatim
geolocator = Nominatim(user_agent='geoapiExercise')

# criando um loop for para preencher os dados
for i in tqdm(range(len(df1))):
    # concatenando lat e long
    query = str(df1.loc[i, 'lat']) + ',' + str(df1.loc[i, 'long'])

    # criando uma variável para receber lat e long
    response = geolocator.reverse(query)

    # verificando a existência e capturando os dados
    if 'place_id' in response.raw:
        df1.loc[i, 'place_id'] = response.raw['place_id']

    if 'osm_type' in response.raw:
        df1.loc[i, 'osm_type'] = response.raw['osm_type']

    if 'country' in response.raw['address']:
        df1.loc[i, 'country'] = response.raw['address']['country']

    if 'country_code' in response.raw['address']:
        df1.loc[i, 'country_code'] = response.raw['address']['country_code']

100%|██████████| 50/50 [00:24<00:00,  2.03it/s]


In [18]:
# verificando o resultado
df1[['id', 'lat', 'long', 'place_id', 'osm_type', 'country', 'country_code']].sample(5)

,id,lat,long,place_id,osm_type,country,country_code
15,9297300055,47.5714,-122.375,148314625,way,United States,us
23,8091400200,47.3533,-122.166,220002194,way,United States,us
34,7955080270,47.4276,-122.157,221459336,way,United States,us
42,7203220400,47.6848,-122.016,169597306,way,United States,us
40,5547700270,47.6145,-122.027,11853353,node,United States,us


### Q4: Adicione os seguintes filtros no mapa:
- Tamanho mínimo da área da sala de estar
- Número mínino de banheiros
- Valor máximo do preço
- Tamanho máximo da área do porão
- Filtro das condições do imóvel
- Filtro por ano de construção

Optei por utilizar diferentes tipos de botões para conhecer melhor a biblioteca [ipywidgets](https://ipywidgets.readthedocs.io/en/stable/).

In [19]:
# definindo o estilo do botão
style = {'description_width': 'initial'}

# criando os filtros
sala_min = widgets.IntSlider(
    value=290,
    min=290,
    max=13540,
    description='Sala de Estar',
    disabled=False,
    orientation='horizontal',
    readout=True,
    style=style)

banheiros_min = widgets.Dropdown(
    options=df['bathrooms'].sort_values().unique().tolist(),
    description='Banheiros',
    disabled=False,
    style=style)

preco_max = widgets.FloatSlider(
    value=7700000.0,
    min=75000.0,
    max=7700000.0,
    description='Preço máximo',
    disabled=False,
    orientation='horizontal',
    readout=True,
    style=style)

porao_max = widgets.IntSlider(
    value=1651359,
    min=520,
    max=1651359,
    description='Porão',
    disabled=False,
    orientation='horizontal',
    redout=True,
    style=style)

condicao_radio = widgets.RadioButtons(
    options=df['condition'].sort_values().unique().tolist(),
    value=3,
    description='Condição do imóvel',
    orientation='horizontal',
    disabled=False,
    style=style)

ano_radio = widgets.IntSlider(
    value=2014,
    min=1900,
    max=2015,
    description='Construção',
    disabled=False,
    orientation='horizontal',
    readout=True,
    style=style)

In [20]:
# criando uma função para plotar o mapa
def mapa_filtro(df, sala, preco, porao, construcao, banheiro, condicao):
    houses = df[(df['sqft_living'] >= sala) &
                (df['bathrooms'] >= banheiro) &
                (df['price'] <= preco) &
                (df['sqft_basement'] <= porao) &
                (df['condition'] == condicao) &
                (df['yr_built'] == construcao)][['id', 'lat', 'long', 'price', 'sqft_living', 'bathrooms', 'sqft_basement', 'condition', 'yr_built']]

    mapa = px.scatter_mapbox(data_frame=houses,
                            lat='lat',
                            lon='long', 
                            hover_data=['sqft_living', 'bathrooms', 'sqft_basement', 'condition', 'yr_built'],
                            color='price',
                            size='price',
                            color_continuous_scale=px.colors.sequential.Viridis_r,
                            size_max=15,
                            zoom=10)

    mapa.update_layout(mapbox_style='open-street-map')
    mapa.update_layout(height=600, margin={'r':0, 't':0, 'l':0, 'b':0})

    mapa.show()

In [21]:
# verificando o resultado da função
widgets.interactive(mapa_filtro, df=fixed(df), 
                    sala=sala_min,
                    banheiro=banheiros_min,
                    preco=preco_max,
                    porao=porao_max, 
                    condicao=condicao_radio,
                    construcao=ano_radio)

interactive(children=(IntSlider(value=290, description='Sala de Estar', max=13540, min=290, style=SliderStyle(…

### Q5: Adicione os seguintes filtros no Dashboard:
- Filtro por data disponível para compra
- Filtro por ano de renovação
- Filtro se possui vista para água ou não

In [22]:
# definindo o estilo dos gráficos
sns.set_theme(style='whitegrid', palette='viridis')
palette = 'viridis'
title = {'size':16, 'color':'#2b2b2b'}
labels = {'size':14, 'color':'#2b2b2b'}
ticks = '#2b2b2b'

# definindo a largura dos filtros
layout = widgets.Layout(width='300px')

# mudando o formato da data
df['year'] = pd.to_datetime(df['date']).dt.strftime('%Y')
df['date'] = pd.to_datetime(df['date']).dt.strftime('%Y-%m-%d')
df['year_week'] = pd.to_datetime(df['date']).dt.strftime('%Y-%U')

# criando o botão para filtrar os dados
data_compra = widgets.Dropdown(
    options=df['date'].sort_values().unique(),
    value='2014-06-23',
    description='Disponível',
    disabled=False,
    style=style,
    layout=layout)

ano_renovacao = widgets.Dropdown(
    options=df['yr_renovated'].sort_values().unique(),
    value=2014,
    description='Renovação',
    disabled=False,
    style=style,
    layout=layout)

vista_agua = widgets.Dropdown(
    options=df['is_waterfront'].unique(),
    description='Vista para a água',
    disabled=False,
    style=style,
    layout=layout)


# criando a função para plotar o dashboard apartir do filtro
def update_dashboard(df, data, renovacao, vista):
    df = df[(df['date'] <= data) &
            (df['yr_renovated'] <= renovacao) &
            (df['is_waterfront'] == vista)]

    # criando o dashboard
    fig = plt.figure(figsize=(20, 12))
    specs = gridspec.GridSpec(nrows=2, ncols=2, figure=fig, hspace=0.3, wspace=0.2, width_ratios=[2, 1])
    fig.tight_layout()

    ax1 = fig.add_subplot(specs[0, :])
    ax2 = fig.add_subplot(specs[1, 0])
    ax3 = fig.add_subplot(specs[1, 1])

    # primeiro gráfico
    by_day = df[['id', 'date']].groupby('date').mean().reset_index()
    sns.lineplot(data=by_day, x='date', y='id', ax=ax1)
    ax1.set_title('Imóveis disponíveis para compra por dia', fontdict=title, pad=20)
    ax1.set_xlabel('')
    ax1.set_ylabel('')
    ax1.tick_params(colors=ticks)
    ax1.set_xticks([df['date'].min(), data])
    ax1.xaxis.set_major_locator(plt.MaxNLocator(10))
    ax1.ticklabel_format(style='plain', axis='y', useOffset=False)
    ax1.grid(axis='x')
    sns.despine(left=True)

    # segundo gráfico
    by_week = df[['price', 'year_week']].groupby('year_week').mean().reset_index()
    sns.barplot(data=by_week, x='year_week', y='price', ax=ax2, palette=palette)
    ax2.set_title('Imóveis disponíveis para compra por semana', fontdict=title, pad=20)
    ax2.set_xlabel('')
    ax2.set_ylabel('')
    ax2.tick_params(colors=ticks)
    ax2.tick_params(rotation=90, axis='x')
    ax2.ticklabel_format(style='plain', axis='y', useOffset=False)

    # terceiro gráfico
    by_year = df[['price', 'year']].groupby('year').sum().reset_index()
    sns.barplot(data=by_year, x='year', y='price', ax=ax3)
    ax3.set_title('Imóveis disponíveis para compra por ano', fontdict=title, pad=20)
    ax3.set_xlabel('')
    ax3.set_ylabel('')
    ax3.tick_params(colors=ticks)
    ax3.ticklabel_format(style='plain', axis='y', useOffset=False)

    plt.show()

widgets.interactive(update_dashboard, df=fixed(df), data=data_compra, renovacao=ano_renovacao, vista=vista_agua)

interactive(children=(Dropdown(description='Disponível', index=52, layout=Layout(width='300px'), options=('201…